In [204]:
import matplotlib.pyplot as plt
import scipy
import sympy
import numpy as np
import control 
import os
import array_to_latex as a2l
from ControlTheoryToolbox import System, CToolbox
import cvxpy

In [205]:
def get_K(system : System, alpha: float, mu :float = None, x0:np.array = None, precision : float = 0.0000001) -> tuple:
    
    P = cvxpy.Variable(system.A.shape)
    Y = cvxpy.Variable((system.B.T).shape)

    constraints = [P >= precision*np.eye(system.A.shape[0]), (P @ (system.A.T) + system.A @ P + 2 * alpha * P + (Y.T) @ (system.B.T) + system.B @ Y) <= -precision*np.eye(system.A.shape[0])]

    # if mu != None and x0 != None:
    #     # constraint on output
    #     constraints.append(np.block([[P, x0], [x0.T, np.ones(1)]]) >= 0)
    #     constraints.append(np.block([[P, Y.T], [Y, mu*mu*np.ones(Y.shape[0])]]) >= 0)
    # print(np.linalg.pinv(P).shape)
    # print((Y@np.linalg.pinv(P)).shape)
    problem = cvxpy.Problem(cvxpy.Minimize(cvxpy.sum_squares(P)), constraints)
    print(problem.solve())
    print(P.value)
    P = P.value
    Y = Y.value
    print((P @ (system.A.T) + system.A @ P + 2 * alpha * P + (Y.T) @ (system.B.T) + system.B @ Y) )
    return Y@np.linalg.pinv(P)

In [206]:
A = np.array([
    [0, 1],
    [0, 0]
])
B = np.array([
    [0],
    [1]
])
system = System(A, B, A*0, B*0)

In [207]:
K = get_K(system, 0.5)

1.6813748271933146e-14
[[ 4.86131128e-08 -4.93065025e-08]
 [-4.93065025e-08  9.79196158e-08]]
[[-4.99998922e-08 -7.21239820e-17]
 [-7.21239687e-17 -1.59996157e-07]]


In [208]:
K

array([[-4.77390274, -3.72083092]])

In [209]:
np.linalg.eig(A+B@K)[0]

array([-1.86041546+1.14575611j, -1.86041546-1.14575611j])